<a href="https://colab.research.google.com/github/Victoooooor/ErgoChairML/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [ ]:
#@title
!apt-get install libmagic-dev
!pip install git+https://github.com/Victoooooor/ErgoChairML.git
!pip install -q imageio
!pip install tensorflow-io
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse

# LOGIN

In [2]:
from google.colab import auth
auth.authenticate_user()

# Load trained model

In [ ]:
bucket_name = 'ergo_chair_ml'
!gsutil -m cp -r gs://{bucket_name}/* ./

# Init

In [ ]:
from gen_chair import pix2pix

from gen_chair import coco
from gen_chair.gen_multi import Preprocess
from gen_chair import pix2pix

from google.colab import files
import shutil
import os
import tqdm
import cv2
import tensorflow as tf
from PIL import Image, ImageSequence
import numpy as np
import tensorflow_io as tfio 

class InferenceConfig(coco.CocoConfig):
  # Set batch size to 1 since we'll be running inference on
  # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

def Infer(preproc, generator, origin, masked = True):

  pre = preproc(origin)
  if type(pre) is tuple:
      if masked:
        seg = pre[0]
      else:
        seg = pre[1]
  else:
      seg = None
  origin = tf.cast(tf.image.resize_with_pad(origin, 256, 256), dtype=tf.int32)

  if seg is not None:
    seg  = tf.keras.preprocessing.image.img_to_array(seg)

  else:
    seg = origin

  seg = tf.expand_dims(seg, axis=0)
  seg = tf.cast(tf.image.resize_with_pad(seg, 256, 256), dtype=tf.int32)

  gen = generator(seg ,training=True)
  gen = tf.keras.utils.array_to_img(gen[0])

  seg = tf.keras.utils.array_to_img(seg[0])

  origin = tf.keras.utils.array_to_img(origin)
  dst = Image.new('RGB', (origin.width + seg.width + gen.width, origin.height))
  dst.paste(origin, (0, 0))
  dst.paste(seg, (origin.width, 0))
  dst.paste(gen, (origin.width+seg.width, 0))
  return dst

imgdr = 'data/'

prep = Preprocess(InferenceConfig)

cpdir_mask = 'masked_checkpoints'
masked = pix2pix(cpdir_mask)
masked.loadcp()

cpdir_ske = 'skeleton_checkpoints'
skeleton = pix2pix(cpdir_ske)
skeleton.loadcp()

# Upload Files

The tab under will prompt to upload local files,
Support Format:
  most image format
  MP4, AVI
  zip

In [ ]:
uploaded = files.upload()
img_suffix = ('.jpg', '.png', '.ico', '.gif', '.jpeg','.jfif')
vid_suffix = ('.avi','.mp4')
filenames = uploaded.keys()

for filename in filenames:
  if filename.endswith('.zip'):
    !unzip {filename} -d "data"
    os.remove(filename)
  elif filename.endswith(img_suffix) or filename.endswith(vid_suffix):
    os.makedirs("data/", exist_ok=True)
    shutil.move(filename, "data/")

# Process & Download

In [ ]:
image_names = sorted([n for n in os.listdir(imgdr) if not n.startswith('.')])

outdr1 = 'out_seg'
!mkdir -p {outdr1}
outdr2 = 'out_ske'
!mkdir -p {outdr2}
audiofile = '_sound.mp3'

for image_name in tqdm.tqdm(image_names):
  image_path = os.path.join(imgdr, image_name)
  #loop through for gif
  if image_path.endswith('.gif'):
    image = Image.open(image_path)
    image.seek(0)
    mask_frames = []
    ske_frames = []
    durations = []
    for frame in ImageSequence.Iterator(image):
        durations.append(image.info['duration'])
        frame = frame.convert('RGB')
        tfframe = tf.convert_to_tensor(np.array(frame))
        mframe = Infer(prep.img_seg, masked.generator, tfframe, True)
        sframe = Infer(prep.img_seg, skeleton.generator, tfframe, False)

        mask_frames.append(mframe)
        ske_frames.append(sframe)
    mask_frames[0].save(os.path.join(outdr1, image_name),
        save_all=True,
        append_images=mask_frames[1:],
        duration=durations,
        loop=0)
    ske_frames[0].save(os.path.join(outdr2, image_name),
        save_all=True,
        append_images=ske_frames[1:],
        duration=durations,
        loop=0)
  #simple image process
  elif image_path.endswith(img_suffix):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image)
    channel = image.shape[-1]
    if channel == 1:
      image = tf.image.grayscale_to_rgb(image)
    elif channel == 4:
      image = tfio.experimental.color.rgba_to_rgb(image)
    elif channel == 2:
      print("Stop, Get Some Help, Only RGB, RGBA, Greyscale")
      continue
    maskout = Infer(prep.img_seg, masked.generator, image, True)
    skeletonout = Infer(prep.img_seg, skeleton.generator, image, False)
    maskout.save(os.path.join(outdr1, image_name))
    skeletonout.save(os.path.join(outdr2, image_name))
  #loop through video frames
  elif image_path.endswith(vid_suffix):
    print('video',image_path)
    video = cv2.VideoCapture(image_path)
    if not video.isOpened():
      print('video does not exist')
      continue
    fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_num = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = 256*3
    frame_height = 256
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    name1 = os.path.join(outdr1, image_name)
    name2 = os.path.join(outdr2, image_name)
    video_writer1 = cv2.VideoWriter(name1,fourcc,fps,(frame_width,frame_height))
    video_writer2 = cv2.VideoWriter(name2,fourcc,fps,(frame_width,frame_height))
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    counter =0
    while True:
      ret, frame = video.read()
      if ret == True:
          tfframe= tf.convert_to_tensor(frame)
          mframe = Infer(prep.img_seg, masked.generator, tfframe, True)
          sframe = Infer(prep.img_seg, skeleton.generator, tfframe, False)
          m_cv2 = cv2.cvtColor(np.array(mframe), cv2.COLOR_RGB2BGR)
          s_cv2 = cv2.cvtColor(np.array(sframe), cv2.COLOR_RGB2BGR)
          video_writer1.write(m_cv2)
          video_writer2.write(s_cv2)
          counter +=1
      if ret == False:
          break
    video.release()
    video_writer1.release()
    video_writer2.release()
    cv2.destroyAllWindows()

!zip -r file.zip {outdr1} {outdr2}
files.download('file.zip')

In [40]:
!rm -rf {outdr1} {outdr2} {imgdr}